In [ ]:
from google.colab import userdata
hf_token = userdata.get('HUGGINGFACE_TOKEN')
from huggingface_hub import login
login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install langchain_community langchainhub chromadb langchain sentence-transformers peft FlagEmbedding bitsandbytes nemoguardrails nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of nemoguardrails to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of nemoguardrails to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip

In [ ]:
from langchain.schema.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
import pandas as pd
import uuid
import re
import json
import nest_asyncio
import os
nest_asyncio.apply()

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# LLM


In [ ]:
import torch
import transformers

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_config = transformers.AutoConfig.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.2,
    max_new_tokens=512,
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

<ipython-input-9-b18d252f13dc>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=generate_text)


# Loading data


In [ ]:
! pip install langchain unstructured[all-docs] pydantic lxml langchainhub fastapi kaleido uvicorn

In [ ]:
!sudo apt-get install poppler-utils tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  poppler-utils tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,002 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 5,002 kB in 1s (5,149 kB/s)
debconf: unable to initialize frontend: Dial

In [ ]:
!pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import os
dir_path = "/content/drive/MyDrive/AI4E/NLP_RAG_Project/data"
pdf_paths = [os.path.join(dir_path, file) for file in os.listdir(dir_path) if file.endswith(".pdf")]

In [ ]:
from lxml import html
from pydantic import BaseModel
from typing import Any, Optional
from unstructured.partition.pdf import partition_pdf
from tqdm import tqdm

# Get elements
all_elements = []
for pdf_path in tqdm(pdf_paths, desc="Processing: "):
  elements = partition_pdf(filename=pdf_path,
                          extract_images_in_pdf=False,
                          infer_table_structure=True,
                          strategy='hi_res',
            )
  all_elements.append(elements)

Processing: 100%|██████████| 3/3 [28:21<00:00, 567.20s/it]


In [ ]:
category_counts = {}

for elements in all_elements:
  for element in elements:
      category = str(type(element))
      if category in category_counts:
          category_counts[category] += 1
      else:
          category_counts[category] = 1

unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.Title'>": 209,
 "<class 'unstructured.documents.elements.Text'>": 99,
 "<class 'unstructured.documents.elements.NarrativeText'>": 1897,
 "<class 'unstructured.documents.elements.Header'>": 2,
 "<class 'unstructured.documents.elements.Table'>": 19,
 "<class 'unstructured.documents.elements.ListItem'>": 35,
 "<class 'unstructured.documents.elements.FigureCaption'>": 4,
 "<class 'unstructured.documents.elements.Image'>": 1}

In [ ]:
class Element(BaseModel):
    type: str
    text: Any

# Categorize by type
categorized_elements = []
text_types = ["<class 'unstructured.documents.elements.Title'>",
        "<class 'unstructured.documents.elements.Text'>",
        "<class 'unstructured.documents.elements.NarrativeText'>"]
for elements in all_elements:
  for element in elements:
      if "unstructured.documents.elements.Table" in str(type(element)):
          categorized_elements.append(Element(type="table", text=str(element)))
      elif str(type(element)) in text_types:
          categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

19
2205


In [ ]:
import json
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/table_elements.json', 'w') as f:
    json.dump([e.dict() for e in table_elements], f, indent=4, ensure_ascii=False)

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/text_elements.json', 'w') as f:
    json.dump([e.dict() for e in text_elements], f, indent=4, ensure_ascii=False)

# Document Splitting


## Text splitting

In [ ]:
import json
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/data/text_elements.json', 'r') as f:
  text_elements = json.load(f)

In [ ]:
all_sentences = [element["text"]  for element in text_elements]

In [ ]:
len(all_sentences)

2205

In [ ]:
import re
# Loại bỏ các phần tử trích xuất lỗi
def contains_letters(text):
    # Tìm kiếm chữ cái trong chuỗi
    pattern = re.compile(r'[a-zA-Z]')
    return bool(pattern.search(text))
new_all_sentences = []
for sent in all_sentences:
  if len(sent) > 10 and contains_letters(sent):
    new_all_sentences.append(sent)
all_sentences = new_all_sentences

In [ ]:
len(all_sentences)

2047

### Semantic Chunking

In [ ]:
sentences = [{'sentence': x, 'index': i} for i, x in enumerate(all_sentences)]
sentences[10: 12]

[{'sentence': '2. Tổ chức quy định tại khoản 1 Điều này gồm:', 'index': 10},
 {'sentence': 'a) Cơ quan nhà nước có hành vi vi phạm mà hành vi đó không thuộc nhiệm vụ quản lý nhà nước được giao;',
  'index': 11}]

In [ ]:
def combine_sentence(sentences, buffer_size=1):
  """
  Function to combine current, pre and next sentence

  buffer_size: distance
  """
  for i in range(len(sentences)):
    combined_sentence = ''
    for j in range(i - buffer_size, i):
      if j >= 0:
        combined_sentence += sentences[j]['sentence'] + ' '
    combined_sentence += sentences[i]['sentence']
    for j in range(i + 1, i + 1 + buffer_size):
      if j < len(sentences):
        combined_sentence += ' ' + sentences[j]['sentence']
    sentences[i]['combined_sentence'] = combined_sentence
  return sentences

sentences = combine_sentence(sentences)

In [ ]:
!pip install -U -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 9.2 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('keepitreal/vietnamese-sbert')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

...

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = embed_model.encode([x['combined_sentence'] for x in sentences])

In [ ]:
for i, sentence in enumerate(sentences):
  sentence['combined_sentence_embedding'] = embeddings[i]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def calculate_cosine_distance(sentences):
  distances = []
  for i in range(len(sentences) - 1):
    embedding_cur = sentences[i]['combined_sentence_embedding']
    embedding_next = sentences[i + 1]['combined_sentence_embedding']
    similarity = cosine_similarity([embedding_cur], [embedding_next])[0][0]
    distance = 1 - similarity
    distances.append(distance)
    sentences[i]['distance_to_next'] = distance
  return distances, sentences

distances, sentences = calculate_cosine_distance(sentences)

In [ ]:
import numpy as np
#Phân vị
breakpoint_percentile_threshold = 80
breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold)
num_distances_above_theshold = len([x for x in distances if x > breakpoint_distance_threshold])

indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold]

In [ ]:
start_index = 0

chunks = []

for index in indices_above_thresh:
    end_index = index

    group = sentences[start_index:end_index + 1]
    combined_text = ' '.join([d['sentence'] for d in group])
    chunks.append(combined_text)

    start_index = index + 1

if start_index < len(sentences):
    combined_text = ' '.join([d['sentence'] for d in sentences[start_index:]])
    chunks.append(combined_text)

In [ ]:
text_documents = [Document(page_content=chunk) for chunk in chunks]

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/text_documents.json', 'w') as f:
    json.dump([e.dict() for e in text_documents], f, indent=4, ensure_ascii=False)

### Chunk with title

In [ ]:
all_chunks = []
i = 0
while i < len(all_sentences):
  first_sentence = all_sentences[i]
  chunk = first_sentence
  if(first_sentence.startswith("Điều")):
    # Duyệt thêm các câu tiếp theo
    i += 1
    while i < len(all_sentences):
      # Break nếu len(chunk) > 1000 hoặc sentence là Điều tiếp theo
      if(len(chunk) > 1000) or all_sentences[i].startswith("Điều "):
        break
      chunk += " " + all_sentences[i]
      i += 1
  # Bắt đầu bằng khoản
  elif re.match(r'^\d+\.\s', first_sentence):
    # TÌm kiếm Điều của khoản này
    for j in range(i - 1, -1, -1):
      if all_sentences[j].startswith("Điều"):
        chunk = all_sentences[j] + " " + chunk + " "
        break
    # Duyệt thêm các câu tiếp theo
    i += 1
    while i < len(all_sentences):
      # Break nếu len(chunk) > 1000 hoặc sentence là 1 Điều, khoản khác
      if(len(chunk) > 1000) or all_sentences[i].startswith("Điều ") or re.match(r'^\d+\.\s', all_sentences[i]):
        break
      chunk += all_sentences[i] + " "
      i += 1
  elif re.match(r'^[a-zđ]\)\s', first_sentence):
    # Tìm kiếm Điều, khoản của ý này
    for j in range(i - 1, -1, -1):
      if re.match(r'^\d+\.\s', all_sentences[j]):
        chunk = all_sentences[j] + " " + chunk + " "
        break
    for k in range(i - 1, -1, -1):
      if all_sentences[k].startswith("Điều"):
        chunk = all_sentences[k] + " " + chunk + " "
        break
    # Duyệt thêm các câu tiếp theo
    i += 1
    while i < len(all_sentences):
      # Break nếu len(chunk) > 1000 hoặc sentence là 1 Điều, khoản tiếp theo
      if(len(chunk) > 1000) or all_sentences[i].startswith("Điều ") or re.match(r'^\d+\.\s', all_sentences[i]):
        break
      chunk += all_sentences[i] + " "
      i += 1
  else:
    # Các thông tin bên lề
    while i < len(all_sentences):
      if(all_sentences[i].startswith("Điều ") or re.match(r'^\d+\.\s', all_sentences[i]) or re.match(r'^[a-zđ]\)\s', all_sentences[i])):
        break
      chunk += all_sentences[i] + " "
      i += 1
  all_chunks.append(chunk)

In [ ]:
len(all_chunks)

496

In [ ]:
text_documents = [Document(page_content=chunk) for chunk in all_chunks]

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/data/title_text_documents.json', 'w') as f:
    json.dump([e.dict() for e in text_documents], f, indent=4, ensure_ascii=False)

## Table

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/table_elements.json', 'r') as f:
  table_elements = json.load(f)

In [ ]:
tables = [element["text"]  for element in table_elements]

In [ ]:
table_documents = [Document(page_content=table) for table in tables]

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/table_documents.json', 'w') as f:
    json.dump([e.dict() for e in table_documents], f, indent=4, ensure_ascii=False)

In [ ]:
table_documents[0].page_content

'ĐƠN VỊ TÍNH HẠNG GIẤY PHÉP LÁI XE Hạng A1 Hạng A2 Hạng A3, A4 1 Pháp luật giao thông đường bộ giờ 8 16 28 2 Cấu tạo và sửa chữa thông thường giờ - - 4 3 Nghiệp vụ vận tải giờ - - 4 4 Kỹ thuật lái xe giờ 2 4 4 Thực hành lái xe giờ 2 12 40 5 Số giờ học thực hành lái xe/học viên Số km thực hành lái xe/học viên giờ km 2 - 12 - 8 60 Số học viên/1 xe tập lái học viên - - 5 6 Số giờ/học viên/khóa đào tạo giờ 12 32 48 7 Tổng số giờ một khóa đào tạo giờ 12 32 80 THỜI GIAN ĐÀO TẠO 1 Số ngày thực học ngày 2 4 10 2 Số ngày nghỉ lễ, khai bế giảng ngày - - 1 3 Cộng số ngày/khóa học ngày 2 4 11'

## Multiple-choice question

In [ ]:
import json
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/data/crawl_mcq_data/crawled_mcqs.json', 'r', encoding='utf-8') as fIn:
  mcq_data = json.load(fIn)

In [ ]:
len(mcq_data)

279

In [ ]:
sublist = [mcq_data[i:i+3] for i in range(0, len(mcq_data), 3)]
mcq_documents = [Document(page_content=str(doc)) for doc in sublist]

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/mcq_documents.json', 'w') as f:
    json.dump([e.dict() for e in mcq_documents], f, indent=4, ensure_ascii=False)

In [ ]:
len(mcq_documents[0].page_content)

1336

# Indexing

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
store = LocalFileStore("./cache/")

embed_model_id = "keepitreal/vietnamese-sbert"

embedder = CacheBackedEmbeddings.from_bytes_store(
    HuggingFaceEmbeddings(model_name=embed_model_id),
    store,
    namespace=embed_model_id,
)

<ipython-input-10-67b5c6696177>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  HuggingFaceEmbeddings(model_name=embed_model_id),


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Summary table

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/table_documents.json', 'r') as f:
    table_dict = json.load(f)
table_documents = [Document(page_content=doc["page_content"]) for doc in table_dict]

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AI4E/NLP_RAG_Project/table_documents.json'

In [ ]:
table_documents[5].page_content

In [ ]:
table_prompt = PromptTemplate(
    input_variables = ["text"],
    template = """Bạn là trợ lý rất giỏi được giao nhiệm vụ tóm tắt bảng bằng Tiếng Việt.\n

    ## Hướng dẫn ##
    Đoạn văn bản biểu diễn bảng theo từng hàng, hãy suy nghĩ theo từng bước, trong đó kí hiệu '-' coi như 0.

    ## Ví dụ 1 ##
    Bảng: "ĐƠN VỊ TÍNH HẠNG GIẤY PHÉP LÁI XE Hạng A1 Hạng A2 Hạng A3, A4 Pháp luật giao thông đường bộ giờ 8 16 28 Cấu tạo và sửa chữa thông thường giờ - - 4 Nghiệp vụ vận tải giờ - - 4 Kỹ thuật lái xe giờ 2 4 4 Thực hành lái xe giờ 2 12 40 Số giờ học thực hành lái xe/học viên giờ 2 12 8 Số km thực hành lái xe/học viên  - - 60 Số học viên/1 xe tập lái học viên - - 5 Số giờ/học viên/khóa đào tạo giờ 12 32 48 Tổng số giờ một khóa đào tạo giờ 12 32 80 THỜI GIAN ĐÀO TẠO Số ngày thực học ngày 2 4 10 Số ngày nghỉ lễ, khai bế giảng ngày - - 1 Cộng số ngày/khóa học ngày 2 4 11"
    Bảng tóm tắt: "Pháp luật giao thông đường bộ: hạng A1: 8 giờ; hạng A2: 16 giờ; hạng A3, A4: 28 giờ.\n Cấu tạo và sửa chữa thông thường: hạng A1: 0 giờ; hạng A2: 0 giờ; hạng A3, A4: 4 giờ.\n Nghiệp vụ vận tải: hạng A1: 0 giờ; hạng A2: 0 giờ; hạng A3, A4: 4 giờ.\n Kỹ thuật lái xe: hạng A1: 2 giờ; hạng A2: 4 giờ; hạng A3, A4: 4 giờ.\n Thực hành lái xe: hạng A1: 2 giờ; hạng A2: 12 giờ; hạng A3, A4: 40 giờ.\n Số giờ học thực hành lái xe / 1 học viên: hạng A1: 2 giờ; hạng A2: 12 giờ; hạng A3, A4: 8 giờ.\n Số km thực hành lái xe của 1 học viên: hạng A1: 0 km; hạng A2: 0 km; hạng A3, A4: 60 km.\n Số học viên/1 xe tập lái: hạng A1: 0 học viên; hạng A2: 0 học viên; hạng A3, A4: 5 học viên.\n Số giờ một khóa đào tạo một học viên: hạng A1: 12 giờ; hạng A2: 32 giờ; hạng A3, A4: 48 giờ.\n Tổng số giờ một khóa đào tạo: hạng A1: 12 giờ; hạng A2: 32 giờ,; hạng A3, A4: 80 giờ.\n Sô ngày thực học: hạng A1: 2 ngày; hạng A2: 4 ngày; hạng A3, A4: 10 ngày.\n Số ngày nghỉ lễ, khai bế giảng: hạng A1: 0 ngày; hạng A2: 0 ngày; hạng A3, A4: 1 ngày.\n Tổng số ngày trong khóa học: hạng A1: 2 ngày; hạng A2: 4 ngày; hạng A3, A4: 11 ngày." \n

    ## Ví dụ 2 ##
    Bảng: "NỘI DUNG ĐƠN VỊ TÍNH B1 (số tự động) lên B1 B1 lên B2 HẠNG GIẤY PHÉP LÁI XE B2 lên C C lên D D lên E B2, D, E lên F C, D, E lên FC B2 lên D C lên E Ôn và kiểm tra kết thúc khóa học ngày 1 2 2 2 2 2 2 2 2 2 Số ngày thực học ngày 15 12 24 24 24 24 34 42 42 Số ngày nghỉ lễ, khai giảng, bế giảng ngày 2 2 4 4 4 4 4 8 8 Số ngày/khóa học ngày 18 16 30 30 30 30 40 52 52"
    Bảng tóm tắt: "Ôn tập và kiểm tra kết thúc khóa học: B1(số tự động) lên B1: 1 ngày; B1 lên B2: 2 ngày; B2 lên C: 2 ngày; C lên D: 2 ngày; D lên E: 2 ngày; B2, D, E lên F: 2 ngày; C, D, E lên FC: 2 ngày; B2 lên D: 2 ngày; C lên E: 2 ngày.\n Số ngày thực học: B1(số tự động) lên B1: 15 ngày; B1 lên B2: 12 ngày; B2 lên C: 24 ngày; C lên D: 24 ngày; D lên E: 24 ngày; B2, D, E lên F: 24 ngày; C, D, E lên FC: 34 ngày; B2 lên D: 42 ngày; C lên E: 42 ngày.\n Số ngày nghỉ lễ, khai giảng, bế giảng: B1(số tự động) lên B1: 2 ngày; B1 lên B2: 2 ngày; B2 lên C: 4 ngày; C lên D: 4 ngày; D lên E: 4 ngày, B2, D, E lên F: 4 ngày; C, D, E lên FC: 4 ngày; B2 lên D: 8 ngày; C lên E: 8 ngày.\n Tổng số ngày trong khóa học: B1(số tự động) lên B1: 18 ngày; B1 lên B2: 4 ngày; B2 lên C: 16 ngày; C lên D: 30 ngày; D lên E: 30 ngày; B2, D, E lên F: 30 ngày; C, D, E lên FC: 40, B2 lên D: 52 ngày; C lên E: 52 ngày." \n

    ## Ví dụ 3 ##
    Bảng: "SỐ TT NỘI DUNG ĐƠN VỊ TÍNH HẠNG GIẤY PHÉP LÁI XE Hạng B1 Hạng B2 Hạng C Học xe số Học xe số tự động cơ khí Số km thực hành lái xe trên sân tập lái/01 học viên km 290 290 290 275 Số km thực hành lái xe trên đường giao thông/01 học viên km 710 810 810 825 Tổng số km thực hành lái xe/01 học viên km 1000 1100 1100 1100"
    Bảng tóm tắt: "Số km thực hành lái xe trên sân tập lái/01 học viên: hạng B1(học xe số tự động): 290km; hạng B1(học xe số cơ khí): 290km; hạng B2: 290km; hạng C: 275km\n\n Số km thực hành lái xe trên đường giao thông/01 học viên:  hạng B1(học xe số tự động): 710km; hạng B1(học xe số cơ khí): 810km; hạng B2: 810km; hạng C: 825km\n\n Tổng số km thực hành lái xe/01 học viên: hạng B1(học xe số tự động): 1000km; hạng B1(học xe số cơ khí): 1100km; hạng B2: 1100km; hạng C:100km" \n

    Tôi sẽ tặng cho bạn 100$.
    Bây giờ, nhiệm vụ của bản là đưa ra một bản tóm tắt về bảng sau: {text}
    Câu trả lời:
    """
)
table_chain = load_summarize_chain(llm, prompt=table_prompt, chain_type="stuff")

In [ ]:
table_summaries = []
for table in table_documents[:6]:
  # Get an ID
  unique_id = str(uuid.uuid4())

  chunk_summary = table_chain.run([table])
  chunk_summary_document = Document(page_content=chunk_summary, metadata={"doc_id": unique_id})
  table_summaries.append(chunk_summary_document)

  table.metadata["doc_id"] = unique_id
print (f"You have {len(table_summaries)} summaries to go along with your {len(table_documents)} chunks")

<ipython-input-15-886386481c85>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chunk_summary = table_chain.run([table])
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packa

You have 6 summaries to go along with your 10 chunks


In [ ]:
idx = table_summaries[5].page_content.find("Câu trả lời:")
print(table_summaries[5].page_content[idx:])

Câu trả lời:
    Ôn tập và kiểm tra kết thúc khóa học: B1(số tự động) lên B1: 1 ngày; B1 lên B2: 2 ngày; B2 lên C: 2 ngày; C lên D: 2 ngày; D lên E: 2 ngày; B2, D, E lên F: 2 ngày; C, D, E lên FC: 2 ngày; B2 lên D: 2 ngày; C lên E: 2 ngày.
    Số ngày thực học: B1(số tự động) lên B1: 15 ngày; B1 lên B2: 12 ngày; B2 lên C: 24 ngày; C lên D: 24 ngày; D lên E: 24 ngày; B2, D, E lên F: 24 ngày; C, D, E lên FC: 34 ngày; B2 lên D: 42 ngày; C lên E: 42 ngày.
    Số ngày nghỉ lễ, khai giảng, bế giảng: B1(số tự động) lên B1: 2 ngày; B1 lên B2: 2 ngày; B2 lên C: 4 ngày; C lên D: 4 ngày; D lên E: 4 ngày, B2, D, E lên F: 4 ngày; C, D, E lên FC: 4 ngày; B2 lên D: 8 ngày; C lên E: 8 ngày.
    Tổng số ngày trong khóa học: B1(số tự động) lên B1: 18 ngày; B1 lên B2: 4 ngày; B2 lên C: 16


## Attach id to text

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/data/semantic_text_documents.json', 'r') as f:
    text_dict = json.load(f)
text_documents = [Document(page_content=doc["page_content"]) for doc in text_dict]

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/data/title_text_documents.json', 'r') as f:
    text_dict = json.load(f)
text_documents = [Document(page_content=doc["page_content"]) for doc in text_dict]

In [ ]:
text_with_ids = []
for doc in text_documents:
  # Get an ID
  unique_id = str(uuid.uuid4())

  text_with_id = Document(page_content=doc.page_content, metadata={"doc_id": unique_id})
  text_with_ids.append(text_with_id)

  doc.metadata["doc_id"] = unique_id

## Attach id to MCQ

In [ ]:
with open('/content/drive/MyDrive/AI4E/NLP_RAG_Project/data/mcq_documents.json', 'r') as f:
    mcq_dict = json.load(f)
mcq_documents = [Document(page_content=doc["page_content"]) for doc in mcq_dict]

In [ ]:
len(mcq_documents)

93

In [ ]:
mcq_with_ids = []
for mcq in mcq_documents:
  # Get an ID
  unique_id = str(uuid.uuid4())

  mcq_with_id = Document(page_content=mcq.page_content, metadata={"doc_id": unique_id})
  mcq_with_ids.append(mcq_with_id)

  mcq.metadata["doc_id"] = unique_id

## Storage

In [ ]:
vectorstore = Chroma(collection_name="summaries", embedding_function=embedder)

docstore = InMemoryStore()

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key="doc_id",
    search_kwargs={"k": 16}
)

<ipython-input-16-4259f2e5f841>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(collection_name="summaries", embedding_function=embedder)


In [ ]:
retriever.vectorstore.add_documents(table_summaries)
retriever.docstore.mset([(x.metadata["doc_id"], x) for x in table_documents])

In [ ]:
retriever.vectorstore.add_documents(text_with_ids)
retriever.docstore.mset([(x.metadata["doc_id"], x) for x in text_documents])

In [ ]:
retriever.vectorstore.add_documents(mcq_with_ids)
retriever.docstore.mset([(x.metadata["doc_id"], x) for x in mcq_documents])

In [ ]:
retrieved_docs = retriever.get_relevant_documents("Lái ô tô sử dụng rượu bia thì bị xử phạt hành chính thế nào?")
print(len(retrieved_docs))
retrieved_docs

16


<ipython-input-18-30d56dfe5202>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents("Lái ô tô sử dụng rượu bia thì bị xử phạt hành chính thế nào?")


[Document(metadata={'doc_id': 'a1a8274c-3aaf-4853-9432-09b5534cb188'}, page_content='Điều 5. Xử phạt người điều khiển xe ô tô và các loại xe tương tự xe ô tô vi phạm quy tắc giao thông đường bộ 10. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: a) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở; b) Không chấp hành yêu cầu kiểm tra về nồng độ cồn của người thi hành công vụ; c) Điều khiển xe trên đường mà trong cơ thể có chất ma túy; d) Không chấp hành yêu cầu kiểm tra về chất ma túy của người thi hành công vụ. '),
 Document(metadata={'doc_id': 'ec8b1bf2-980a-4b7d-91db-a63399a00782'}, page_content='Điều 7. Xử phạt người điều khiển máy kéo, xe máy chuyên dùng (sau đây gọi chung là xe) vi phạm quy tắc giao thông đường bộ 9. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với người điều khiển xe thực hiện một trong 

# Retrieval Merchanims

## Multi-queri & Re-ranked

In [ ]:
multi_queries_template = """
  Bạn là 1 chuyên gia về việc tạo ra các câu hỏi liên quan nhất đến 1 câu hỏi cho trước bằng Tiếng Việt.\n

  ## Hướng dẫn 1 ## \n
  Nếu Truy vấn không chỉ rõ phương tiện nào, làm rõ Truy vấn liên quan đến loại
  phương tiện giao thông nào trong các loại phương tiện giao thông sau: \n
  1.Xe ô tô \n2.Xe gắn máy \n3.Máy kéo, xe máy chuyên dụng. \n4.Xe đạp, xe thô sơ. \n 5.Người đi bộ. \n\n

  ## Ví dụ hướng dẫn 1 ## \n
  Chỉ sinh ra 3 truy vấn liên quan nhất. \n
  Truy vấn cho trước: "Nếu tôi lái xe có sử dụng rượu bia thì bị phạt bao nhiêu?"
  3 Truy vấn liên quan nhất đến truy vấn trên là:" 1. Nếu tôi lái xe ô tô có sử dụng rượu bia thì bị phạt bao nhiêu?.\n
  2. Nếu tôi lái xe máy có sử dụng rượu bia thì bị phạt bao nhiêu?.\n
  3. Nếu tôi lái xe đạp, xe thô sơ có sử dụng rượu bia thì bị phạt bao nhiêu?.\n"
  ##
  Bây giờ, chỉ tạo ra truy vấn, không thêm thông tin nào khác,
  hãy tạo ra 3 truy vấn liên quan nhất với 1 truy vấn đầu vào duy nhất:

  {question}
  Chỉ tạo ra đúng 3 truy vấn theo hướng dẫn 1, không sinh ra thêm thông tin, trả lời nào khác.
  Câu trả lời(3 truy vấn liên quan nhất):
"""
prompt_multi_queries = PromptTemplate(
    input_variables=["question"],
    template=multi_queries_template
)

In [ ]:
generate_queries = (
    prompt_multi_queries
    | llm
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [ ]:
generate_queries.invoke("Nếu đi ngược chiều trong đường 1 chiều có vi phạm pháp luật?")

['  1. Nếu đi ngược chiều trong đường 1 chiều trong đường xe ô tô thì có vi phạm pháp luật?',
 '  2. Nếu đi ngược chiều trong đường 1 chiều trong đường xe gắn máy thì có vi phạm pháp luật?',
 '  3. Nếu đi ngược chiều trong đường 1 chiều trong đường xe đạp, xe thô sơ thì có vi phạm pháp luật?']

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('namdp-ptit/ViRanker', use_fp16=True)

In [ ]:
def rerank_multi_documents(results, question):
  scores = {}
  for docs in results:
    for doc in docs:
      doc_str = doc.page_content
      if doc_str not in scores:
        similarity_score = reranker.compute_score([question, doc_str], normalize=True)
        scores[doc_str] = similarity_score
  reranked_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
  reranked_results = [(doc, score) for doc, score in reranked_scores]
  return reranked_results

## Decomposition

In [ ]:
decomposition_template = """
          Bạn là 1 chuyên gia trong việc tạo ra các câu hỏi phụ
      liên quan nhất đến 1 câu hỏi đầu vào.\n
      Mục đích là chia nhỏ câu hỏi đầu vào thành các vấn đề phụ
      để trả lời từng vấn đề dẫn đến trả lời câu hỏi chính. \n
      Bạn phải tạo nhiều truy vấn liên quan đến: {question}. \n
      Output(3 truy vấn):
"""
prompt_decomposition = PromptTemplate(
    input_variables=["question"],
    template=decomposition_template
)

In [ ]:
generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [ ]:
question = "Các cơ sở đào tạo các hạng giấy phép lái xe được sử dụng tài liệu nào??"
questions = generate_queries_decomposition.invoke(question)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
questions

['',
 '          Bạn là 1 chuyên gia trong việc tạo ra các câu hỏi phụ',
 '      liên quan nhất đến 1 câu hỏi đầu vào.',
 '',
 '      Mục đích là chia nhỏ câu hỏi đầu vào thành các vấn đề phụ',
 '      để trả lời từng vấn đề dẫn đến trả lời câu hỏi chính. ',
 '',
 '      Bạn phải tạo nhiều truy vấn liên quan đến: Các cơ sở đào tạo các hạng giấy phép lái xe được sử dụng tài liệu nào??. ',
 '',
 '      Output(3 truy vấn):',
 '',
 '      1. Các cơ sở đào tạo các hạng giấy phép lái xe được sử dụng tài liệu nào để học hỏi nội dung cơ bản về lái xe?',
 '',
 '      2. Các cơ sở đào tạo các hạng giấy phép lái xe được sử dụng tài liệu nào để học hỏi nội dung về quy định về an toàn trên đường bộ?',
 '',
 '      3. Các cơ sở đào tạo các hạng giấy phép lái xe được sử dụng tài liệu nào để học hỏi nội dung về quy định về an toàn trên đường bộ đường sắt?']

In [ ]:
answer_template = """ Đây là câu hỏi chính mà bạn phải trả lời:
        \n --- \n {question} \n --- \n

        Đây là các cặp câu hỏi phụ và câu trả lời liên quan nhất đến câu hỏi trên:
        \n --- \n {qa_pairs} \n --- \n

        Đây là ngữ cảnh liên quan đến câu hỏi chính:
        \n --- \n {context} \n --- \n

        Sử dụng ngữ cảnh và các cặp câu hỏi phụ + câu trả lời liên quan nhất đến câu hỏi
        chính để trả lời cho câu hỏi: {question}
"""
prompt = ChatPromptTemplate.from_template(answer_template)
def format_qa_pair(question, answer):
  """ Format Q&A pair"""
  formatted_string = f"Câu hỏi: {question}\nCâu trả lời: {answer}\n\n"
  return formatted_string.strip()

qa_pairs = ""
for q in questions:
  rag_chain = (
      {
          "context": itemgetter("question") | retriever,
          "question": itemgetter("question"),
          "qa_pairs": itemgetter("qa_pairs")
      }
      | prompt
      | llm
      | StrOutputParser()
  )
  answer = rag_chain.invoke({"question": q, "qa_pairs": qa_pairs})
  qa_pair = format_qa_pair(q, answer)
  qa_pairs += "\n --- \n" + qa_pair

In [ ]:
answer

## Re-rank

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('namdp-ptit/ViRanker', use_fp16=True)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/796 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [ ]:
def rerank_documents(results, question):
  reranked_results = []
  for doc in results:
    doc_str = doc.page_content
    similarity_score = reranker.compute_score([question, doc_str], normalize=True)
    reranked_results.append((doc_str, similarity_score))
  reranked_documents = sorted(reranked_results, key=lambda x: x[1], reverse=True)
  # reranked_results = [document for document in reranked_documents if document[1][0] > 0.6]
  return reranked_documents[:5]

In [ ]:
question = "Thi sát hạch thực hành  lái xe hạng A1, A2 phải làm những gì?"
retrieved_docs = retriever.get_relevant_documents(question)
rerank_documents(retrieved_docs, question)

<ipython-input-22-e0fe2b85e3e3>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(question)


[('Điều 21. Nội dung và quy trình sát hạch lái xe 3. Nội dung sát hạch cấp giấy phép lái xe b) Sát hạch thực hành lái xe trong hình đối với các hạng A1, A2  Người dự sát hạch phải điều khiển xe mô tô qua 04 bài sát hạch: đi theo hình số 8, qua vạch đường thẳng, qua đường có vạch cản, qua đường gồ ghề. c) Sát hạch thực hành lái xe trong hình đối với các hạng A3, A4 Người dự sát hạch phải điều khiển xe tiến qua hình chữ chi và lùi theo hướng ngược lại. d) Sát hạch thực hành lái xe trong hình đối với các hạng B1, B2, C, D và E Người dự sát hạch phải thực hiện đúng trình tự và điều khiển xe qua các bài sát hạch đã bố trí tại trung tâm sát hạch như: xuất phát, dừng xe nhường đường cho người đi bộ, dừng và khởi hành xe trên dốc, qua vệt bánh xe và đường vòng vuông góc, qua ngã tư có đèn tín hiệu điều khiển giao thông, qua đường vòng quanh co, ghép xe vào nơi đỗ (hạng B1, B2 và C thực hiện ghép xe dọc; hạng B1, B2 và D, E thực hiện ghép xe ngang), tạm dừng ở chỗ có đường sắt chạy qua, thao tá

# Guardrails

In [ ]:
from langchain.llms.huggingface_endpoint import HuggingFaceEndpoint
from nemoguardrails import LLMRails
from nemoguardrails import RailsConfig
class Instruction:
    def __init__(self, type, content):
        self.type = type
        self.content = content

# Load configuration from the specified path
config = RailsConfig.from_path("/content/drive/MyDrive/AI4E/NLP_RAG_Project/config")

# Add instructions and sample conversations to the configuration
config.instructions = [
    Instruction(
        type="general",
        content="Đây là cuộc trò chuyện của của người dùng và bot. Bot có kiến thức về Luật giao thông đường bộ Việt Nam và các thông tin liên quan kì thi sát hạch lái xe. Bot được thiết kế để trả lời các câu hỏi của người dùng về chủ đề này."
    )
]

config.sample_conversation = [
    {
        "user": "Xin chào. Bạn có thể giúp tôi điều gì?",
        "bot": "Xin chào bạn! Tôi có thể giúp bạn trả lời các câu hỏi liên quan đến Luật giao thông đường bộ và kì thi sát hạch giấy phép lái xe."
    },
    {
        "user": "Luật giao thông đường bộ Việt Nam áp dụng cho những đối tượng nào?",
        "bot": "Luật giao thông đường bộ Việt Nam áp dụng đối với các tổ chức, cá nhân liên quan đến giao thông đường bộ trên lãnh thổ nước Cộng hòa xã hội chủ nghĩa Việt Nam."
    }
]

# Create a HuggingFaceEndpoint object
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
)

# Create an LLMRails object using the configuration and HuggingFaceEndpoint instance
rails = LLMRails(config, llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
messages = [{
    "role": "user",
    "content": "lồn là gì?"
}
]
response = rails.generate(messages=messages)
if(rails.explain().llm_calls[0].completion.startswith(" Có")):
  response = {
      'role': 'assistant',
      'content': "Xin lỗi tôi không thể trả lời điều này."}

In [ ]:
print(response)

{'role': 'assistant', 'content': 'Xin lỗi, tôi không thể trả lời điều này.'}


In [ ]:
info = rails.explain()

In [ ]:
info.print_llm_calls_summary()

Summary: 2 LLM call(s) took 3.68 seconds .

1. Task `self_check_input` took 0.29 seconds .
2. Task `general` took 3.39 seconds .



In [ ]:
print(info.llm_calls[0].completion)

 Có


In [ ]:
print(info.llm_calls[0].prompt)

Nhiệm vụ của bạn là kiểm tra xem tin nhắn của người dùng có tuân thủ chính sách
dưới đây khi thực hiện cuộc trò chuyện không.

Các chính sách cấm:
-  Hỏi các vấn đề không liên quan đến giao thông.
-  Chia sẻ các nội dung nhạy cảm và thông tin cá nhân.
-  Chứa tin nhắn trống.
-  Chứa những dữ liệu có hại.
-  Yêu cầu giả mạo ai đó, giả mạo thông tin nào đó.
-  Sử dụng từ ngữ lăng mạ, không tôn trọng.

Đây là các quy định nghiêm ngặt. Nếu 1 trong các điều cấm trên bị vi phạm,
bạn phải chặn tin nhắn của người dùng bằng cách trả lời "Có".
Nếu không vi phạm điều nào trên đây: Trả lời "Không".

Tin nhắn của người dùng: "Chủ tịch nước là ai?"

Câu hỏi: Tin nhắn của người dùng có nên bị chặn không?
Trả lời [Có/Không]:


# RAG Chain

In [ ]:
template = """Bạn là 1 chuyên gia về Luật giao thông đường bộ Việt Nam và thông tin về đào tạo & thi sát hạch các hạng giấy phép lái xe.
      Bạn phải dựa trên ngữ cảnh, tổng hợp thông tin và đưa ra câu trả lời 1 cách tự nhiên, có lời dẫn cho câu trả lời. \n
      Đặc biệt,không xuyên tạc câu trả lời. \n

      ## Hướng dẫn 1 ##
      Ngữ cảnh sẽ bao gồm các văn bản được sắp xếp giảm dần dựa theo trọng số đi kèm.
      Trọng số này thể hiện mức độ liên quan của câu hỏi và các văn bản. Trọng số càng cao mức độ liên quan càng cao. \n\n
      ## \n\n

      ## Hướng dẫn 2 ##
      Đối với các ngữ cảnh có chứa Câu hỏi, Các lựa chọn và Đáp án đúng thì câu trả lời của bạn không chỉ ra là câu hỏi số mấy trong tài liệu,
      nếu đáp án đúng là "cả ý 1 và ý 2", hãy tổng hợp cả 2 ý lại và trả lời. \n
      # Ví dụ hướng dẫn 2 #
      Câu hỏi: "Có những biện pháp nào để tiết kiệm nhiên liệu cho xe?"
      Ngữ cảnh: "'Câu hỏi': 'Câu 241. Các biện pháp tiết kiệm nhiên liệu khi chạy xe?', 'Các lựa chọn': ['Bảo dưỡng xe theo định kỳ và có kế hoạch lộ trình trước khi chạy xe.', 'Kiểm tra áp suất lốp theo quy định và chạy xe với tốc độ phù hợp với tình trạng mặt đường và mật độ giao thông trên đường.', 'Cả ý 1 và ý 2.'], 'Đáp án đúng': ['Cả ý 1 và ý 2.']"
      Câu trả lời: "Các biện pháp để tiết kiệm nhiên liệu khi chạy xe: Bảo dưỡng xe theo định kỳ và có kế hoạch lộ trình trước khi chạy xe, kiểm tra áp suất lốp theo quy định và chạy xe với tốc độ phù hợp với tình trạng mặt đường và mật độ giao thông trên đường." \n
      ## \n\n

      ## Hưỡng dẫn 3 ##
      Đối với các ngữ cảnh có nhiều mức độ, đều có trọng số liên quan cao đến câu hỏi, trả lời chứa đầy đủ thông tin về mức độ.\n
      # Ví dụ hướng dẫn 3 #
      Câu hỏi: "Lái xe ô tô có sử dụng rượu bia bị xử phạt hành chính như thế nào?"
      Ngữ cảnh: "['Điều 5. Xử phạt người điều khiển xe ô tô và các loại xe tương tự xe ô tô vi phạm quy tắc giao thông đường bộ 8. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: a) Điều khiển xe đi ngược chiều trên đường cao tốc, lùi xe trên đường cao tốc, trừ các xe ưu tiên đang đi làm nhiệm vụ khẩn cấp theo quy định; b) Gây tai nạn giao thông không dừng lại, không giữ nguyên hiện trường, bỏ trốn không đến trình báo với cơ quan có thẩm quyền, không tham gia cấp cứu người bị nạn; c) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở. ', 'Điều 5. Xử phạt người điều khiển xe ô tô và các loại xe tương tự xe ô tô vi phạm quy tắc giao thông đường bộ 10. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: a) Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở; b) Không chấp hành yêu cầu kiểm tra về nồng độ cồn của người thi hành công vụ; c) Điều khiển xe trên đường mà trong cơ thể có chất ma túy; d) Không chấp hành yêu cầu kiểm tra về chất ma túy của người thi hành công vụ. ']
      Câu trả lời: "Lái xe ô tô có sử dụng rượu bia bị xử phạt hành chính với các mức độ như sau: \n
      1. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với người điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở. \n
      2. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với người điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở. \n"
      ## \n\n

      ## Hướng dẫn 4 ##
      Trả lời 1 cách tự nhiên dựa trên ngữ cảnh, không trích dẫn nguyên vẹn từ văn bản. \n
      Không trích dẫn tài liệu tham khảo. \n
      Không trích dẫn thuộc Điều nào trong văn bản. \n
      Không sinh ra thêm thông tin, câu hỏi, lưu ý nào khác.\n
      ## \n\n

      Bây giờ, bạn hãy trả lời câu hỏi dựa trên ngữ cảnh sau:

      {context}

      Yêu cầu: {question}
      Trả lời:
"""
# retrieval_chain = generate_queries | retriever.map() | (lambda results: rerank_multi_documents(results, question))
prompt = ChatPromptTemplate.from_template(template)
def generate_answer(question):
  retrieval_chain = retriever | (lambda results: rerank_documents(results, question))
  final_rag_chain = (
      {"context": retrieval_chain,
      "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )
  answer = final_rag_chain.invoke(question)
  return answer

## Demo

In [ ]:
def demo_rag_chain(question):
  result = generate_answer(question)
  start_answer_idx = result.find("Trả lời:") + len("Trả lời")

  print("\nAnswer: " + result[start_answer_idx:])

In [ ]:
question1 = "“Dải phân cách” trên đường bộ gồm những loại nào?"
demo_rag_chain(question1)

In [ ]:
question2 = "Thi sát hạch thực hành  lái xe hạng A1, A2 phải làm những gì?"
demo_rag_chain(question2)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer: :
      Người dự sát hạch phải điều khiển xe mô tô qua 04 bài sát hạch: đi theo hình số 8, qua vạch đường thẳng, qua đường có vạch cản, qua đường gồ ghề.
      (Trong trường hợp người dự sát hạch làm việc tại các đô thị từ loại 3 trở lên, phải sử dụng thiết bị chấm điểm tự động.)


# Chat

In [ ]:
end_text = ["tạm biệt", "chào tạm biệt"]
def rag_chat(rails):
  user_input = ""
  separated = '-' * 100
  while(user_input.lower() not in end_text):
    if(user_input == ""):
      user_input = input("| You |\n-------\n")
    if(user_input.lower() not in end_text):
      print(f'{separated}\n| Model |\n---------\n')
      messages = [{
        "role": "user",
        "content": f"{user_input}"
      }]
      response = rails.generate(messages=messages)
      if(rails.explain().llm_calls[0].completion.startswith(" Có")):
          response = {
                'role': 'assistant',
                'content': "Xin lỗi, tôi không thể trả lời điều này."}
          print(response['content'])
      else:
        output = generate_answer(user_input)
        start_idx = output.find("Trả lời:") + len("Trả lời")
        result = output[start_idx:]
        print(result)
      user_input = ""
      print(f'{separated}\n')
      user_input = input("| You |\n-------\n")
    else:
      break

In [ ]:
rag_chat(rails)

# Evaluation

In [ ]:
import pandas as pd
eval_dataset = "/content/drive/MyDrive/AI4E/NLP_RAG_Project/evaluation/eval_dataset.csv"
data = pd.read_csv(eval_dataset)
data.head()

,question,ground_truth
0,Đối tượng áp dụng của Luật giao thông đường bộ...,Luật giao thông đường bộ Việt Nam áp dụng đối ...
1,Hệ thống các báo hiệu trên đường bộ gồm những gì?,Hệ thống báo hiệu đường bộ gồm hiệu lệnh của n...
2,“Dải phân cách” trên đường bộ gồm những loại nào?,Dải phân cách gồm loại cố định và loại di động.
3,Khi thấy đèn giao thông màu vàng báo hiệu còn ...,Trong trường hợp đèn giao thông báo hiệu màu v...
4,Nếu trên đường có biển báo hiệu cố định và cả ...,Tại nơi có biển báo hiệu cố định lại có báo hi...


In [ ]:
data['question'][11]

'Sử dụng điện thoại di động khi đang điều khiển xe ô tô chạy trên đường bị xử phạt bao nhiêu tiền?'

In [ ]:
import ast
def get_context(context_w_scores):
  tuple_list = ast.literal_eval(context_w_scores)
  contexts = [item[0] for item in tuple_list]
  return contexts

In [ ]:
from tqdm import tqdm
df = pd.DataFrame({
    "question": [],
     "contexts": [],
     "answer": [],
    "ground_truth": [],
})
limit = 18
for i, row in tqdm(data.iterrows(), total=limit):
  if i >= limit:
    break
  question = row['question']
  ground_truth = row['ground_truth']

  output = generate_answer(question)
  start_answer_idx = output.find("Trả lời:") + len("Trả lời:")
  answer = output[start_answer_idx:].strip()

  contexts_w_scores = []
  start_context_idx = output.find("ngữ cảnh sau:") + len("ngữ cảnh sau:")
  end_context_idx = output.find("Yêu cầu:")
  contexts_w_scores = f'''{output[start_context_idx:end_context_idx].strip()}'''
  contexts = get_context(contexts_w_scores)

  df = pd.concat([df, pd.DataFrame({
      "question": [question],
      "contexts": [contexts],
      "answer": [answer],
      "ground_truth": [ground_truth]
  })], ignore_index=True)

  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  6%|▌         | 1/18 [00:39<11:18, 39.92s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 11%|█         | 2/18 [02:11<18:44, 70.25s/it]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` i

In [ ]:
# df.to_csv('/content/drive/MyDrive/AI4E/NLP_RAG_Project/evaluation/sematic_eval_dataset.csv')

In [ ]:
df.to_csv('/content/drive/MyDrive/AI4E/NLP_RAG_Project/evaluation/title_eval_dataset.csv')

## Ragas

In [ ]:
!pip install ragas

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.7/185.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.0 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
eval_dataset = Dataset.from_dict(df[:2])
eval_dataset

Dataset({
    features: ['question', 'contexts', 'answer', 'ground_truth'],
    num_rows: 2
})

In [ ]:
from ragas import evaluate
result = evaluate(
    eval_dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
)
result = result.to_pandas()

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[5]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}})
ERROR:ragas.executor:Exception raised in Job[7]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}})
ERROR:ragas.executor:Exception raised in Job[2]: RateLimitError(Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platfor

In [ ]:
result

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall
0,Đối tượng áp dụng của Luật giao thông đường bộ...,[Điều 1. Phạm vi điều chỉnh Luật này quy định ...,"""Đối tượng áp dụng của Luật giao thông đường b...",Luật giao thông đường bộ Việt Nam áp dụng đối ...,NaN,NaN,NaN,NaN
1,Hệ thống các báo hiệu trên đường bộ gồm những gì?,[1. Hệ thống báo hiệu đường bộ gồm hiệu lệnh c...,Hệ thống các báo hiệu trên đường bộ gồm hiệu l...,Hệ thống báo hiệu đường bộ gồm hiệu lệnh của n...,NaN,NaN,NaN,NaN


## RAG-Evaluator

In [ ]:
!pip install rag-evaluator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=7cadb2d8afa8651a9f432a84109a04b697c1f50fe26f25585956652c969d6c89
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rag_evaluator import RAGEvaluator
evaluator = RAGEvaluator()

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


### Semantic


In [ ]:
semantic_df = pd.read_csv('/content/drive/MyDrive/AI4E/NLP_RAG_Project/evaluation/semantic_eval_dataset.csv')
semantic_df.head()

,Unnamed: 0,question,contexts,answer,ground_truth
0,0,Đối tượng áp dụng của Luật giao thông đường bộ...,['Điều 1. Phạm vi điều chỉnh Luật này quy định...,"""Đối tượng áp dụng của Luật giao thông đường b...",Luật giao thông đường bộ Việt Nam áp dụng đối ...
1,1,Hệ thống các báo hiệu trên đường bộ gồm những gì?,['1. Hệ thống báo hiệu đường bộ gồm hiệu lệnh ...,Hệ thống các báo hiệu trên đường bộ gồm hiệu l...,Hệ thống báo hiệu đường bộ gồm hiệu lệnh của n...
2,2,Khi thấy đèn giao thông màu vàng báo hiệu còn ...,"['3. Tín hiệu đèn giao thông có ba mầu, quy đị...","Có thể đi tiếp, nhưng phải giảm tốc độ, chú ý ...",Trong trường hợp đèn giao thông báo hiệu màu v...
3,3,Nếu trên đường có biển báo hiệu cố định và cả ...,"[""[{'Câu hỏi': 'Câu 85. Tại nơi có biển báo hi...",Người tham gia giao thông phải chấp hành theo ...,Người tham gia giao thông phải chấp hành hiệu ...
4,4,Trong trường hợp hiệu lệnh của người điều khiể...,"[""[{'Câu hỏi': 'Câu 84. [ĐIỂM LIỆT] Trên đường...",Hiệu lệnh của người điều khiển giao thông mâu ...,trong trường hợp có sự mâu thuẫn giữa hiệu lện...


In [ ]:
metrics_list = []
for i, row in semantic_df.iterrows():
  question = row['question']
  answer = row['answer']
  ground_truth = row['ground_truth']

  metrics = evaluator.evaluate_all(question, answer, ground_truth)
  bleu = metrics['BLEU']
  rouge = metrics['ROUGE-1']
  bert_p = metrics['BERT P']
  bert_r = metrics['BERT R']
  bert_f1 = metrics['BERT F1']
  perplexity = metrics['Perplexity']
  diversity = metrics['Diversity']
  racial_bias = metrics['Racial Bias']

  metrics_list.append({
      "bleu": bleu,
      "rouge": rouge,
      "bert_p": bert_p,
      "bert_r": bert_r,
      "bert_f1": bert_f1,
      "perplexity": perplexity,
      "diversity": diversity,
      "racial_bias": racial_bias
  })


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
semantic_df = pd.concat([semantic_df, pd.DataFrame(metrics_list)], axis=1)

In [ ]:
semantic_df.head(17)

In [ ]:
print(f"Average BLEU: {semantic_df['bleu'].mean()}")
print(f"Average ROUGE: {semantic_df['rouge'].mean()}")
print(f"Average BERT P: {semantic_df['bert_p'].mean()}")
print(f"Average BERT R: {semantic_df['bert_r'].mean()}")
print(f"Average BERT F1: {semantic_df['bert_f1'].mean()}")
print(f"Average Perplexity: {semantic_df['perplexity'].mean()}")
print(f"Average Diversity: {semantic_df['diversity'].mean()}")
print(f"Average Racial Bias: {semantic_df['racial_bias'].mean()}")

Average BLEU: 52.231185733271374
Average ROUGE: 0.7855311896047057
Average BERT P: 0.8551967669935787
Average BERT R: 0.857333523385665
Average BERT F1: 0.8556144027148976
Average Perplexity: 20.86023010927088
Average Diversity: 0.9042304950056939
Average Racial Bias: 0.505683397545534


### Titile


In [ ]:
title_df = pd.read_csv('/content/drive/MyDrive/AI4E/NLP_RAG_Project/evaluation/title_eval_dataset.csv')
title_df.head()

,Unnamed: 0,question,contexts,answer,ground_truth
0,0,Đối tượng áp dụng của Luật giao thông đường bộ...,['Điều 2. Đối tượng áp dụng Luật này áp dụng đ...,"""Đối tượng áp dụng của Luật giao thông đường b...",Luật giao thông đường bộ Việt Nam áp dụng đối ...
1,1,Hệ thống các báo hiệu trên đường bộ gồm những gì?,['Điều 10. Hệ thống báo hiệu đường bộ 1. Hệ th...,Hệ thống các báo hiệu trên đường bộ gồm hiệu l...,Hệ thống báo hiệu đường bộ gồm hiệu lệnh của n...
2,2,“Dải phân cách” trên đường bộ gồm những loại nào?,['Điều 3. Giải thích từ ngữ 10. Dải phân cách ...,"""Dải phân cách gồm loại cố định và loại di động.""",Dải phân cách gồm loại cố định và loại di động.
3,3,Khi thấy đèn giao thông màu vàng báo hiệu còn ...,['Điều 10. Hệ thống báo hiệu đường bộ 3. Tín h...,"Có thể đi tiếp nhưng phải giảm tốc độ, chú ý q...",Trong trường hợp đèn giao thông báo hiệu màu v...
4,4,Nếu trên đường có biển báo hiệu cố định và cả ...,['Điều 11. Chấp hành báo hiệu đường bộ 1. Ngườ...,Người tham gia giao thông phải chấp hành theo ...,Người tham gia giao thông phải chấp hành hiệu ...


In [ ]:
metrics_list = []
for i, row in title_df.iterrows():
  question = row['question']
  answer = row['answer']
  ground_truth = row['ground_truth']

  metrics = evaluator.evaluate_all(question, answer, ground_truth)
  bleu = metrics['BLEU']
  rouge = metrics['ROUGE-1']
  bert_p = metrics['BERT P']
  bert_r = metrics['BERT R']
  bert_f1 = metrics['BERT F1']
  perplexity = metrics['Perplexity']
  diversity = metrics['Diversity']
  racial_bias = metrics['Racial Bias']

  metrics_list.append({
      "bleu": bleu,
      "rouge": rouge,
      "bert_p": bert_p,
      "bert_r": bert_r,
      "bert_f1": bert_f1,
      "perplexity": perplexity,
      "diversity": diversity,
      "racial_bias": racial_bias
  })


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
title_df = pd.concat([title_df, pd.DataFrame(metrics_list)], axis=1)

In [ ]:
title_df.head(17)

,Unnamed: 0,question,contexts,answer,ground_truth,bleu,rouge,bert_p,bert_r,bert_f1,perplexity,diversity,racial_bias
0,0,Đối tượng áp dụng của Luật giao thông đường bộ...,['Điều 2. Đối tượng áp dụng Luật này áp dụng đ...,"""Đối tượng áp dụng của Luật giao thông đường b...",Luật giao thông đường bộ Việt Nam áp dụng đối ...,76.906938,0.933333,0.894604,0.925009,0.909553,19.339602,0.888889,0.498386
1,1,Hệ thống các báo hiệu trên đường bộ gồm những gì?,['Điều 10. Hệ thống báo hiệu đường bộ 1. Hệ th...,Hệ thống các báo hiệu trên đường bộ gồm hiệu l...,Hệ thống báo hiệu đường bộ gồm hiệu lệnh của n...,79.725273,0.967213,0.952771,0.962166,0.957445,15.881882,0.944444,0.505192
2,2,“Dải phân cách” trên đường bộ gồm những loại nào?,['Điều 3. Giải thích từ ngữ 10. Dải phân cách ...,"""Dải phân cách gồm loại cố định và loại di động.""",Dải phân cách gồm loại cố định và loại di động.,83.857668,1.000000,0.922667,0.956366,0.939214,32.574200,0.909091,0.507599
3,3,Khi thấy đèn giao thông màu vàng báo hiệu còn ...,['Điều 10. Hệ thống báo hiệu đường bộ 3. Tín h...,"Có thể đi tiếp nhưng phải giảm tốc độ, chú ý q...",Trong trường hợp đèn giao thông báo hiệu màu v...,47.172176,0.750000,0.929038,0.842775,0.883807,23.826878,0.952381,0.523139
4,4,Nếu trên đường có biển báo hiệu cố định và cả ...,['Điều 11. Chấp hành báo hiệu đường bộ 1. Ngườ...,Người tham gia giao thông phải chấp hành theo ...,Người tham gia giao thông phải chấp hành hiệu ...,54.597097,0.775510,0.930291,0.914929,0.922546,38.698036,0.923077,0.511333
5,5,Trong trường hợp hiệu lệnh của người điều khiể...,"[""[{'Câu hỏi': 'Câu 84. [ĐIỂM LIỆT] Trên đường...",Trong trường hợp hiệu lệnh của người điều khiể...,trong trường hợp có sự mâu thuẫn giữa hiệu lện...,64.437171,0.945736,0.939256,0.925753,0.932456,10.944200,0.777778,0.485450
6,6,Đường ưu tiên là phần đường thế nào?,['Điều 3. Giải thích từ ngữ 15. Đường ưu tiên ...,"""Đường ưu tiên là phần đường mà trên đó phương...",Đường ưu tiên là đường mà trên đó phương tiện ...,85.888862,0.966102,0.948927,0.959690,0.954278,13.313464,0.926829,0.498270
7,7,Không được phép lùi xe ở những khu vực nào?,"['Điều 16. Lùi xe 1. Khi lùi xe, người điều kh...",Không được lùi xe ở những khu vực như:\n ...,"Không được lùi xe ở khu vực cấm dừng, trên phầ...",71.449165,0.935252,0.888568,0.934398,0.910907,12.890912,0.943396,0.518038
8,8,Người điều khiển xe mô tô hai bánh có được chở...,"['Điều 30. Người điều khiển, người ngồi trên x...",Người điều khiển xe mô tô hai bánh không được ...,Người điều khiển xe mô tô hai bánh chỉ được ch...,65.297680,0.915254,0.910295,0.902678,0.906470,24.127022,0.973684,0.509086
9,9,Người có giấy phép lái xe hạng B1 được điều kh...,['Điều 16. Phân hạng giấy phép lái xe 7. Hạng ...,"Ô tô chở người đến 9 chỗ ngồi, kể cả chỗ ngồi ...","Ô tô chở người đến 9 chỗ ngồi, kể cả chỗ ngồi ...",87.405202,0.945205,0.963306,0.937646,0.950303,13.225552,0.812500,0.513977


In [ ]:
print(f"Average BLEU: {title_df['bleu'].mean()}")
print(f"Average ROUGE: {title_df['rouge'].mean()}")
print(f"Average BERT P: {title_df['bert_p'].mean()}")
print(f"Average BERT R: {title_df['bert_r'].mean()}")
print(f"Average BERT F1: {title_df['bert_f1'].mean()}")
print(f"Average Perplexity: {title_df['perplexity'].mean()}")
print(f"Average Diversity: {title_df['diversity'].mean()}")
print(f"Average Racial Bias: {title_df['racial_bias'].mean()}")

Average BLEU: 60.70693391782712
Average ROUGE: 0.8407034280566879
Average BERT P: 0.8868829475508796
Average BERT R: 0.8896282944414351
Average BERT F1: 0.88788241147995
Average Perplexity: 23.972371578216553
Average Diversity: 0.895213199820016
Average Racial Bias: 0.5052244298987918
